# Klasifikacija

Algoritmi koji će se koristiti su algoritmi nadgledanog učenja, preciznije, algoritmi klasifikacije. Cilj je napraviti model koji će uspešno predviđati koji automobili su kick a koji nisu.

Kao metod evaluacije koristićemo unakrsnu validaciju koja je kompleksniji ali i verodostojniji vid ocenjivanja kvaliteta modela od *trening-test* podele. Unakrsna validacija prolazi kroz delove skupa podataka i svaki od njih jedanput uzima za test dok ostale koristi za trening. Za finalnu ocenu se uzima prosek ocena svih delova. Kod *trening-test* podele zbog nasumičnosti podele podataka možemo dobiti dobre ocene modela iako to možda nije slučaj.

Algoritmi klasifikacije koje ćemo koristiti:
 - Stablo odlučivanja
 - Naivni Bajes
 - K najbližih suseda
 - Logistička regresija
 - Ansambl algoritmi (Glasanje, Stacking i Random forest)

Za evaluacionu metriku ćemo uzeti ROC krivu, tj površinom ispod nje (AUC). AUC je korisna evaluciona metrika jer uzima u obzir sve granice odlučivanja i daje nam najbolju.
Od dve vrste greške skuplje će nas koštati ona kada auto koji jeste kick klasifikujemo kao da nije. Ali pošto ne znamo cenu grešaka odnosno dobit pogodaka, ne možemo da koristimo *total_cost* meru. 



In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, validation_curve
from mlxtend.classifier import StackingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import VarianceThreshold, chi2, mutual_info_classif, SelectKBest, RFECV
%matplotlib inline

Učitavamo pripremljene podatke.

In [3]:
df = pd.read_csv('data/pripremljeniPodaci.csv')

In [4]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

## Selekcija atributa

Neki atributi su više značajni od drugih i iz tog razloga ćemo izvršiti selekciju atributa i naš skup podataka svesti samo na najneophodnije. Postoje tri vrste metoda za odabir atributa za modele. Obradićemo filter metode i metodu obavijanja.

### Filter Metoda

Granična varijansa (VarianceThreshold)

In [5]:
selection = VarianceThreshold(0.05)
selection.fit(X)
X_filter_variance_thresh = X.loc[:, selection.get_support()]
#X_filter_variance_thresh

Zajednička informacije (mutual_info_classif)

In [7]:
selection = SelectKBest(k=40, score_func=mutual_info_classif) 
selection.fit(X, y)
X_filter_mutual_info = X.loc[:, selection.get_support()]
X_filter_mutual_info.columns

Index(['VehYear', 'WheelTypeID', 'MMRAcquisitionAuctionCleanPrice',
       'MMRAcquisitionRetailAveragePrice', 'MMRCurrentRetailAveragePrice',
       'MMRCurrentRetailCleanPrice', 'VNZIP1', 'Make_CHEVROLET', 'Make_KIA',
       'Model_CONTINENTAL', 'Model_ELEMENT', 'Model_EXCURSION', 'Model_FUSION',
       'Model_I-290', 'Model_MARINER', 'Model_MAZDA6', 'Model_MIATA',
       'Model_OUTLANDER', 'Model_PROTEGE', 'Model_RAIDER', 'Model_SEBRING',
       'Model_TACOMA', 'Model_TRAILBLAZER', 'Trim_GLE', 'Trim_GLS', 'Trim_Pre',
       'Trim_RT', 'Trim_Z71', 'Trim_ZX4', 'Trim_ZX5', 'Size_VAN',
       'CarBodyStyle_HATCHBACK', 'Perc_Diff_RetAVG', 'Perc_Diff_AuctAVG',
       'Perc_Diff_RetCLEAN', 'Perc_Diff_AuctCLEAN', 'Perc_Diff_RetAuctCLEAN',
       'Perc_Diff_PAST_RetAuctAVG', 'Perc_Diff_PAST_RetAuct_CLEAN',
       'Perc_Diff_PRESENT_RetAuct_CLEAN'],
      dtype='object')

Sa graničnom varijansom 0,05 filter metode smanjili smo broj atributa sa 388 na 42, dok smo sa metodom *zajedničkih informacija* sveli na 40.

### Obavijajuća metoda 

In [6]:
model = LogisticRegression(solver='liblinear')
model_as = RFECV(estimator=model, 
                 min_features_to_select=40, cv=10, step=10, 
                 scoring='roc_auc')
model_as.fit(X, y)
X_rfecv = X.loc[:, model_as.get_support()]

Obavijajućom metodom skup podataka smo sveli na 40 atributa.

## Klasifikacija

Izlistavamo algoritme i pravimo novi dataframe.

In [7]:
alg = ['Stablo odlučivanja', 'Naivni Bajes', 'KNN', 'Log regresija', 
       'Glasanje meko', 'Stacking lr', 'Random forest']
# pravimo df
df_rezultati = pd.DataFrame(alg)
# postavimo Algoritam za index
df_rezultati.columns = ['Algoritam']
df_rezultati = df_rezultati.set_index('Algoritam')
df_rezultati

""
Algoritam
Stablo odlučivanja
Naivni Bajes
KNN
Log regresija
Glasanje meko
Stacking lr
Random forest


In [8]:
model_dt = DecisionTreeClassifier()
model_nb = GaussianNB()
model_knn = KNeighborsClassifier()
model_lr = LogisticRegression()

model_voting_s = VotingClassifier(voting='soft', 
                                  estimators=[('dt', model_dt),
                                              ('nb',model_nb),
                                              ('knn',model_knn),
                                              ('lr', model_lr)])
model_stacking_meta_lr = StackingClassifier(classifiers=[model_dt, model_nb, model_knn, model_lr], 
                                            meta_classifier = model_lr)
model_rf = RandomForestClassifier(n_estimators=100)

modeli = [model_dt, model_nb, model_knn, model_lr, 
          model_voting_s, model_stacking_meta_lr, model_rf]

Radimo unakrsnu validaciju za nova tri skupa podataka.

In [9]:
lista = list()
for model in modeli:
    score = cross_val_score(model, X_filter_variance_thresh, y, cv=10, scoring='roc_auc')
    lista.append(round(np.mean(score)*100, 2))
df_rezultati['X_filter_variance_thresh'] = lista
    
df_rezultati

,X_filter_variance_thresh
Algoritam,
Stablo odlučivanja,53.72
Naivni Bajes,62.06
KNN,52.17
Log regresija,61.39
Glasanje meko,60.86
Stacking lr,57.02
Random forest,64.62


In [10]:
lista = list()
for model in modeli:
    score = cross_val_score(model, X_filter_mutual_info, y, cv=10, scoring='roc_auc')
    lista.append(round(np.mean(score)*100, 2))
df_rezultati['X_filter_mutual_info'] = lista
    
df_rezultati

,X_filter_variance_thresh,X_filter_mutual_info
Algoritam,,
Stablo odlučivanja,53.72,54.31
Naivni Bajes,62.06,62.98
KNN,52.17,55.39
Log regresija,61.39,60.81
Glasanje meko,60.86,60.78
Stacking lr,57.02,55.06
Random forest,64.62,62.83


In [11]:
lista = list()
for model in modeli:
    score = cross_val_score(model, X_rfecv, y, cv=10, scoring='roc_auc')
    lista.append(round(np.mean(score)*100, 2))
df_rezultati['X_rfecv'] = lista
    
df_rezultati

,X_filter_variance_thresh,X_filter_mutual_info,X_rfecv
Algoritam,,,
Stablo odlučivanja,53.72,54.31,54.02
Naivni Bajes,62.06,62.98,63.15
KNN,52.17,55.39,54.79
Log regresija,61.39,60.81,62.20
Glasanje meko,60.86,60.78,62.86
Stacking lr,57.02,55.06,57.58
Random forest,64.62,62.83,65.11


U nastavku ćemo raditi samo sa X_rfecv podacima jer daju najbolje rezultate klasifikacije.

In [12]:
X = X_rfecv

## Optimizacija hiperparametara

U narednom delu ćemo isprobavati različite parametre nekih modela.

### K najbližih suseda

In [21]:
#vrednosti parametara koje treba probati
#knn_params = {'n_neighbors': [1,2,3,4,5,6,7,8,9,10,12,50,100]} 
#knn_params = {'n_neighbors': [100,200,300,400,500,600,700]}
knn_params = {'n_neighbors': [450, 475, 500,525, 550]}
grid_knn_opt = GridSearchCV(model_knn, knn_params, cv=10, scoring='roc_auc')
    
grid_knn_opt.fit(X,y)
print('Best param: ', grid_knn_opt.best_params_)

Best param:  {'n_neighbors': 475}


Optimalan broj suseda je 475.

### Logistička regresija

In [22]:
#logistic_params = {'C':np.linspace(start=0.001, stop=10, num=21)}
#logistic_params = {'C':np.linspace(start=1, stop=1.5, num=20)}
#logistic_params = {'C':np.linspace(start=1, stop=3, num=30)}
logistic_params = {'C':np.linspace(start=1.4, stop=2, num=20)}
grid_lr_opt = GridSearchCV(model_lr, logistic_params, cv=10, scoring='roc_auc')
grid_lr_opt.fit(X,y)

print('Best param: ', grid_lr_opt.best_params_)

Best param:  {'C': 1.5578947368421052}


Optimalna vrednost parametra C je 1.55

### Stablo odlučivanja

In [20]:
max_depths = {'max_depth': np.linspace(3, 10, 8, endpoint=True)}
grid_dt_opt = GridSearchCV(model_dt, max_depths, cv=10, scoring='roc_auc')
grid_dt_opt.fit(X,y)

print('Best param: ', grid_dt_opt.best_params_)

Best param:  {'max_depth': 6.0}


Optimalna dubina stabla je 6.

### Random forest

In [17]:
#forest2 = RandomForestClassifier(n_estimators=10, n_jobs=None, random_state=42, class_weight='balanced')
parameters = {'max_features': [8,10,12], 'min_samples_leaf': [1,2,3],'max_depth': [22, 25, 28]}
#parameters = {'max_features': [10, 15, 20], 'min_samples_leaf': [1,2,3],'max_depth': [35, 45, 55]}
grid_forest_opt = GridSearchCV(model_rf, parameters, cv=5, scoring='accuracy')
grid_forest_opt.fit(X,y)

print('Best param: ', grid_forest_opt.best_params_)

Best param:  {'max_depth': 22, 'max_features': 8, 'min_samples_leaf': 3}


Optimalan minimalan broj uzoraka potrebnih za razdvajanje čvora je 1, dok je maksimalna dubina 25 i maksimalan broj atributa 10.

In [18]:
df_optimizovano = df_rezultati.iloc[[0,2,3,6], [2]]
df_optimizovano

,X_rfecv
Algoritam,
Stablo odlučivanja,54.02
KNN,54.79
Log regresija,62.20
Random forest,65.11


In [24]:
lista_posle_optimizacije = list()
for model in [grid_dt_opt, grid_knn_opt, grid_lr_opt, grid_forest_opt]:
    scores = cross_val_score(model, X, y, cv=10, scoring='roc_auc')
    lista_posle_optimizacije.append(round(np.mean(scores)*100, 2))

df_optimizovano["posle optimizacije"] = lista_posle_optimizacije

In [25]:
df_optimizovano.columns = ['pre optimizacije', 'posle optimizacije']
df_optimizovano

,pre optimizacije,posle optimizacije
Algoritam,,
Stablo odlučivanja,54.02,62.89
KNN,54.79,60.96
Log regresija,62.20,62.28
Random forest,65.11,66.26


Nakon optimizacije hiperparametara uviđamo napredak u AUC oceni naših modela.